In [1]:
from kedro.runner.sequential_runner import SequentialRunner
from kedro.io.data_catalog import DataCatalog

In [2]:
from jaffle_shop.utilities.context import all_logging_disabled

In [3]:
default_pipeline = pipelines['__default__']
default_pipeline.datasets()


{
    'stg_customers',
    'orders',
    'raw_orders',
    'stg_orders',
    'raw_payments',
    'raw_customers',
    'customers',
    'params:payment_methods',
    'stg_payments'
}

In [4]:
default_pipeline = pipelines['__default__']

with all_logging_disabled():
    free_outputs = SequentialRunner().run(
        pipeline=default_pipeline.filter(
            tags=None,
            from_nodes = None,
            to_nodes = None,
            node_names = None,
            from_inputs = None,
            to_outputs = None,
            node_namespace = None,
        ),
        catalog=DataCatalog(
            feed_dict={
                x: catalog.load(x).head(0)
                if 'params:' not in x else catalog.load(x)
                for x in default_pipeline.inputs()

            }
        ),
     #   hook_manager=
    )

        

In [5]:
from jaffle_shop.utilities.graph import ibis_expr_to_graph
nx_ibis_graphs = {k:ibis_expr_to_graph(v) for k,v in free_outputs.items()}
nx_ibis_graphs


{
    'customers': <networkx.classes.digraph.DiGraph object at 0x15789bd00>,
    'orders': <networkx.classes.digraph.DiGraph object at 0x1578957b0>
}

In [6]:
from jaffle_shop.utilities.graph import kedro_pipeline_to_graph

In [7]:
nx_kedro_graph = kedro_pipeline_to_graph(default_pipeline, catalog)

In [8]:
nx_kedro_graph 

In [9]:
#default_pipeline.nodes[0].run({x:catalog.load(x).head(0).cache() for x in default_pipeline.nodes[0].inputs})

In [12]:
from jaffle_shop.utilities.graph import render_nx_graph

render_nx_graph(nx_kedro_graph, color_field="attr_type").show('graph.html')

graph.html
graph.html


<IPython.lib.display.IFrame object at 0x15707e500>

In [11]:
render_nx_graph(
    nx_ibis_graphs['orders'], 
    color_field="operation_name", 
    label_attr="operation_name",
    title_attr="data_name"
).show('graph.html')

graph.html
graph.html


<IPython.lib.display.IFrame object at 0x157982c20>